In [1]:
import numpy as np
from scipy.integrate import quad
def erf_asy(x):
    a1, a2, a3, a4, a5 = 0.254829592, -0.284496736, 1.421413741, -1.453152027, 1.061405429
    p = 0.3275911
    sign = np.sign(x) 
    x = np.abs(x)
    t = 1 / (1 + p * x)
    y = 1 - (((((a5 * t + a4) * t + a3) * t + a2) * t + a1) * t * np.exp(-x * x))
    return sign * y



import numpy as np

def phi(x):
    return 0.5 * (1 + np.erf(x / np.sqrt(2)))

from scipy.special import erfinv

def phi_inv(p):
    return np.sqrt(2) * erfinv(2*p - 1)


def my_integrand(z, rho, A):
    den = np.sqrt(2*(1 - rho**2)) + 1e-8  # avoid zero divide
    temp = (A - np.sqrt(2)*rho*z) / den
    return np.exp(-z**2) * erf_asy(temp)

def Integrat_quad(a, b, rho, A):
    f = lambda z: my_integrand(z, rho, A)
    val ,error= quad(f, a, b,limit=10000)
    return val


def f1(x):
    return np.exp(-x**2)*erf_asy((A-rho*x*np.sqrt(2))/np.sqrt(2*(1-rho**2)))


# print(erf_asy(0))

In [2]:
#Bootstrapping
import pandas as pd

df=pd.read_csv('/home/dherya.malhotra/Carpe_DM/PAPER_CODES/MBZUAI_POC_PAPER_DATA - Sheet1.csv')
df=df[df['Stock market return (%, year-on-year)(Y)']!=0]
df = df[df['Oil Export Price (X)($/Barrel)'].notna()]
A=df[["Oil Export Price (X)($/Barrel)","Stock market return (%, year-on-year)(Y)","GDP per Capita (Constant 2015 US$)(Z)","Political Risk Index(Z)($)"]]
A.shape




(110, 4)

In [3]:

def Bootstrap(A):
    r=A.shape[0]
    c=A.shape[1]
    ans=pd.DataFrame(columns=A.columns)
    for i in range(r):
        temp=np.random.randint(0,r)
        ans.loc[len(ans)]=A.iloc[temp]
    return ans

In [4]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm


from scipy.integrate import quad

def Frechet_Bound_PNS_PN_PS(A,iter,alpha,y,x,x_):
    PNS_lower_bound=[]
    PNS_upper_bound=[]
    PN_lower_bound=[]
    PN_upper_bound=[]
    PS_lower_bound=[]
    PS_upper_bound=[]
    for i in range(iter):
        B=Bootstrap(A)
        II_y = (B['Stock market return (%, year-on-year)(Y)'] < y).astype(int)
        X = np.array(B['Oil Export Price (X)($/Barrel)']).reshape(-1, 1)
        C1 = np.array(B['GDP per Capita (Constant 2015 US$)(Z)']).reshape(-1, 1)
        C2 = np.array(B['Political Risk Index(Z)($)']).reshape(-1, 1)
        clf = LogisticRegression(max_iter=500)
        clf.fit(np.hstack([X, C1, C2]), II_y)
        X0_input = np.hstack([np.ones_like(C1)*x, C1, C2])
        X1_input = np.hstack([np.ones_like(C1)*x_, C1, C2])
        P0 = clf.predict_proba(X0_input)[:, 1]
        P1 = clf.predict_proba(X1_input)[:, 1]
        ANS_lb=np.maximum(0,P0-P1)
        PNS_lower_bound_B=ANS_lb.mean()
        PNS_lower_bound.append(PNS_lower_bound_B)
        PN_lower_bound.append((ANS_lb/(1-P1)).mean())
        PS_lower_bound.append((ANS_lb/P0).mean())
        ANS_ub=np.minimum(P0,1-P1)
        PNS_upper_bound.append(ANS_ub.mean())
        PN_upper_bound.append((ANS_ub/(1-P1)).mean())
        PS_upper_bound.append((ANS_ub/P0).mean())
    PNS_lower_bound=np.array(PNS_lower_bound)
    PNS_upper_bound=np.array(PNS_upper_bound)
    PN_lower_bound=np.array(PN_lower_bound)
    PN_upper_bound=np.array(PN_upper_bound)
    PS_lower_bound=np.array(PS_lower_bound)
    PS_upper_bound=np.array(PS_upper_bound)
    PNS_lower_bound_CI_lb=np.percentile(PNS_lower_bound,alpha)
    PNS_lower_bound_mean=np.mean(PNS_lower_bound)
    PNS_lower_bound_CI_ub=np.percentile(PNS_lower_bound,100-alpha)
    PNS_upper_bound_CI_lb=np.percentile(PNS_upper_bound,alpha)
    PNS_upper_bound_mean=np.mean(PNS_upper_bound)
    PNS_upper_bound_CI_ub=np.percentile(PNS_upper_bound,100-alpha)
    PN_lower_bound_CI_lb=np.percentile(PN_lower_bound,alpha)
    PN_lower_bound_mean=np.mean(PN_lower_bound)
    PN_lower_bound_CI_ub=np.percentile(PN_lower_bound,100-alpha)
    PN_upper_bound_CI_lb=np.percentile(PN_upper_bound,alpha)
    PN_upper_bound_mean=np.mean(PN_upper_bound)
    PN_upper_bound_CI_ub=np.percentile(PN_upper_bound,100-alpha)
    PS_lower_bound_CI_lb=np.percentile(PS_lower_bound,alpha)
    PS_lower_bound_mean=np.mean(PS_lower_bound)
    PS_lower_bound_CI_ub=np.percentile(PS_lower_bound,100-alpha)
    PS_upper_bound_CI_lb=np.percentile(PS_upper_bound,alpha)
    PS_upper_bound_mean=np.mean(PS_upper_bound)
    PS_upper_bound_CI_ub=np.percentile(PS_upper_bound,100-alpha)
    return np.array([PNS_lower_bound_CI_lb,PNS_lower_bound_mean,PNS_lower_bound_CI_ub,PNS_upper_bound_CI_lb,PNS_upper_bound_mean,PNS_upper_bound_CI_ub,PN_lower_bound_CI_lb,PN_lower_bound_mean,PN_lower_bound_CI_ub,PN_upper_bound_CI_lb,PN_upper_bound_mean,PN_upper_bound_CI_ub,PS_lower_bound_CI_lb,PS_lower_bound_mean,PS_lower_bound_CI_ub,PS_upper_bound_CI_lb,PS_upper_bound_mean,PS_upper_bound_CI_ub])
        



In [5]:
tem=Frechet_Bound_PNS_PN_PS(A,1000,2.5,0,60,80)

In [11]:
print('PNS_LB_CI',tem[0],tem[1],tem[2])

PNS_LB_CI 0.033076053039491314 0.10838249095288816 0.19020590111561891


In [12]:
print('PNS_UB_CI',tem[3],tem[4],tem[5])

PNS_UB_CI 0.3337449821016895 0.4226473064537623 0.508197380764199


In [13]:
print('PN_LB_CI',tem[6],tem[7],tem[8])

PN_LB_CI 0.05275909488705003 0.1735532272622753 0.3023077163170176


In [14]:
print('PN_UB_CI',tem[9],tem[10],tem[11])

PN_UB_CI 0.48044338286446364 0.6911340393951603 0.8662789815563303


In [15]:
print('PS_LB_CI',tem[12],tem[13],tem[14])

PS_LB_CI 0.08022480834853504 0.24904647474619043 0.42385148456164895


In [16]:
print('PS_UB_CI',tem[15],tem[16],tem[17])

PS_UB_CI 0.8327870249600751 0.9533588990148851 1.0
